In [1]:
import os
import numpy as np
import cv2
from glob import glob
from tqdm import tqdm
from sklearn.model_selection import train_test_split
from albumentations import HorizontalFlip, VerticalFlip,Rotate
from skimage import io, color, filters, restoration
import numpy as np
import matplotlib.pyplot as plt
from skimage import io, filters

In [2]:

def create_dir(path):
    """ Create a directory. """
    if not os.path.exists(path):
        os.makedirs(path)

In [3]:

import numpy as np
import matplotlib.pyplot as plt
from skimage import io, filters

def anisotropic_diffusion(x):
    x = x.astype(float)
    gradient_x = filters.sobel_h(x)
    gradient_y = filters.sobel_v(x)
    structure_tensor = np.array([[gradient_x**2, gradient_x * gradient_y],
                             [gradient_x * gradient_y, gradient_y**2]])
    iterations = 10
    delta_t = 0.1
    denoised_image = np.copy(x)
    kappa=30
    for _ in range(iterations):
        gradient_x = filters.sobel_h(denoised_image)
        gradient_y = filters.sobel_v(denoised_image)
        gradient_magnitude = np.sqrt(gradient_x**2 + gradient_y**2)
        diffusion = np.exp(-kappa * gradient_magnitude)
        delta_x = np.roll(denoised_image, -1, axis=0) - denoised_image
        delta_y = np.roll(denoised_image, -1, axis=1) - denoised_image
        divergence = delta_x + delta_y
        denoised_image += delta_t * diffusion * divergence
        x=denoised_image
    return x
    
    

In [4]:


def load_data(path, split=0.15):
    """ Load the images and masks """
    images = sorted(glob(f"{path}/*/images/*.png"))
    masks = sorted(glob(f"{path}/*/masks/*.png"))

    """ Split the data """
    split_size = int(len(images) * split)
    train_x, valid_x = train_test_split(images, test_size=split_size, random_state=42)
    train_y, valid_y = train_test_split(masks, test_size=split_size, random_state=42)

    return (train_x, train_y), (valid_x, valid_y)

In [5]:
clahe = cv2.createCLAHE(clipLimit=2.0, tileGridSize=(8, 8))
kernel = np.ones((5, 5), np.uint8)

# Apply CLAHE to the image


In [6]:



def pre_process(images, masks, save_path, augment=True):
    """ Performing data augmentation. """
    H = 128
    W = 128

    for idx, (x, y) in tqdm(enumerate(zip(images, masks)), total=len(images)):
        """ Extracting the dir name and image name """
        dir_name = x.split("\\")[-3]
        name = dir_name + "_" + x.split("\\")[-1].split(".")[0]

        """ Read the image and mask """
        x = cv2.imread(x, cv2.IMREAD_GRAYSCALE)
        x=cv2.GaussianBlur(x, (5,5), 0)
       # x=anisotropic_diffusion(x)
        #x = cv2.cvtColor(x, cv2.COLOR_BGR2GRAY)
#         x=cv2.equalizeHist(x)
          #x=cv2.fastNlMeansDenoising(x, None, h=10, templateWindowSize=7, searchWindowSize=21)
        #x = cv2.bilateralFilter(x, d=9, sigmaColor=75, sigmaSpace=75)
        
        x = clahe.apply(x)
        # Apply dilation
        #x = cv2.dilate(x, kernel, iterations=1)
        # Apply erosion
        #x = cv2.erode(x, kernel, iterations=1)
        y = cv2.imread(y, cv2.IMREAD_GRAYSCALE)

        if augment == True:
            aug = HorizontalFlip(p=1)
            augmented = aug(image=x, mask=y)
            x1 = augmented["image"]
            y1 = augmented["mask"]

            aug = VerticalFlip(p=1)
            augmented = aug(image=x, mask=y)
            x2 = augmented['image']
            y2 = augmented['mask']

            aug = Rotate(limit=45, p=1)
            augmented = aug(image=x, mask=y)
            x3 = augmented["image"]
            y3 = augmented["mask"]

            X = [x, x1, x2,x3]
            Y = [y, y1, y2,y3]

        else:
            X = [x]
            Y = [y]
        

        idx = 0
        for i, m in zip(X, Y):
            i = cv2.resize(i, (W, H))
            m = cv2.resize(m, (W, H))
#             m = m/255.0
#             m = (m > 0.5) * 255

            if len(X) == 1:
                tmp_image_name = f"{name}.jpg"
                tmp_mask_name  = f"{name}.jpg"
            else:
                tmp_image_name = f"{name}_{idx}.jpg"
                tmp_mask_name  = f"{name}_{idx}.jpg"

            image_path = os.path.join(save_path, "image/", tmp_image_name)
            mask_path  = os.path.join(save_path, "mask/", tmp_mask_name)
    

            cv2.imwrite(image_path, i)
            cv2.imwrite(mask_path, m)


            idx += 1


In [ ]:
""" Load the dataset """
dataset_path = os.path.join(r"data_raw/train")
dataset_path

'data_train_test/train'

In [8]:
(train_x, train_y), (valid_x, valid_y) = load_data(dataset_path, split=0.15)

In [9]:


print("Train: ", len(train_x))
print("Valid: ", len(valid_x))


Train:  539
Valid:  94


In [10]:

create_dir("data/train/image")
create_dir("data/train/mask/")
create_dir("data/valid/image/")
create_dir("data/valid/mask/")


In [11]:
pre_process(train_x, train_y, "data/train/", augment=True)
pre_process(valid_x, valid_y, "data/valid/", augment=True)

100%|██████████| 94/94 [00:01<00:00, 67.05it/s]
